In [2]:
#### from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import os
import time
import unidecode

from numpy import array
from numpy import argmax
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import math
import operator
import warnings
warnings.filterwarnings('ignore')
import random

import requests
from requests.structures import CaseInsensitiveDict
import json

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from keras.layers import Activation, Dense
from tensorflow.keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

<h2> 1. Passing Data<h2>

<h5> Load PBP Data - Passing / QB <h5>

In [12]:

df_PBP_Passing = pd.read_csv("Defense/passing_pbp_coverage_OPP_081022.csv")

<h5> Grouping <h5>

In [13]:
group = df_PBP_Passing.groupby(by=['team',"gameId", "playerId","playerName", 'oppTeamId','nickName']).sum()

group.reset_index(inplace=True)
group = group[['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName','FP_total']]

<h3> Load Passing by RouteType <h3>

In [14]:
passingRouteType = pd.read_csv("Split/Passing_RouteType_2020_0817.csv")
#passingRouteType = pd.read_csv("Split/Passing_RouteType0817.csv")
passingRouteType['route'] = passingRouteType['route'].astype(int)
routeTypes = np.unique(passingRouteType['route'])

for i in routeTypes:
    a = passingRouteType[passingRouteType['route'] == i][['playerName','FP/DB']].reset_index(drop = True)
    a = a.rename(columns={'FP/DB' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'], how = 'left')

<h3> Load Passing by DropType <h3>

In [15]:
passingDropType = pd.read_csv("Split/Passing_DropType_2020_0817.csv")
#passingDropType = pd.read_csv("Split/Passing_DropType0817.csv")
dropType = np.unique(passingDropType['DropType_descr'])

for i in dropType:
    a = passingDropType[passingDropType['DropType_descr'] == i][['playerName','FP/DB']].reset_index(drop = True)
    a = a.rename(columns={'FP/DB' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'], how = 'left')


<h3> Load Passing by CoverageType  <h3>

In [16]:
passingCoverageType = pd.read_csv("Split/Passing_CoverageType_2020_0817.csv")
#passingCoverageType = pd.read_csv("Split/Passing_CoverageType0817.csv")
coverageType = np.unique(passingCoverageType['CoverageType_descr'])

for i in coverageType:
    a = passingCoverageType[passingCoverageType['CoverageType_descr'] == i][['playerName','FP/DB']].reset_index(drop = True)
    a = a.rename(columns={'FP/DB' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'], how = 'left')
    

<h3> Load Defense FP/play <h3>

In [17]:
#passingRouteType_def = pd.read_csv("Split/Passing_RouteTyoe_def_0817.csv")
passingRouteType_def = pd.read_csv("Split/Passing_RouteTyoe_def_2020_0817.csv")
passingRouteType_def['route'] = passingRouteType_def['route'].astype(int)
targetType = np.unique(passingRouteType_def['route'])


for i in targetType:
    a = passingRouteType_def[passingRouteType_def['route'] == i][['nickName','FP/DB']].reset_index(drop = True)
    split = str(i) + "_def"
    a = a.rename(columns={'FP/DB' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    
    

In [18]:

#passingDropType_def = pd.read_csv("Split/Passing_DropType_def_0817.csv")
passingDropType_def = pd.read_csv("Split/Passing_DropType_def_2020_0817.csv")
dropType = np.unique(passingDropType_def['DropType_descr'])


for i in dropType:
    a = passingDropType_def[passingDropType_def['DropType_descr'] == i][['nickName','FP/DB']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/DB' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    


In [19]:

passingCoverageType_def = pd.read_csv("Split/Passing_CoverageType_def_2020_0817.csv")
coverageType = np.unique(passingCoverageType_def['CoverageType_descr'])


for i in coverageType:
    a = passingCoverageType_def[passingCoverageType_def['CoverageType_descr'] == i][['nickName','FP/DB']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/DB' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

In [20]:

passingCoverageType_def = pd.read_csv("Split/Passing_CoverageType_def_pooled_2020_0817.csv")
coverageType = np.unique(passingCoverageType_def['CoverageType_pooled'])


for i in coverageType:
    a = passingCoverageType_def[passingCoverageType_def['CoverageType_pooled'] == i][['nickName','FP/DB']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/DB' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

In [22]:
# Merge with position
roster = pd.read_csv("roster.csv", index_col = False)

group = pd.merge(group, roster[['playerId', 'positionCategory']], left_on = ['playerId'],  #2
                          right_on=['playerId'], how = 'left')


<h3> Train Test Split <h3>

In [24]:
from sklearn.model_selection import train_test_split
import missingno as msno

In [252]:
# Drop columns
groupSplit = group.copy()

In [253]:
train, test = train_test_split(groupSplit, test_size=0.2)

trainNum = train.drop(columns = ['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName'])
testNum = test.drop(columns = ['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName'])

X_train = trainNum.drop(['FP_total'], axis = 1)
y_train = trainNum['FP_total']
X_test = testNum.drop(['FP_total'], axis = 1)
y_test = testNum['FP_total']

In [254]:
# Normalization
def normalize(raw_data):
    return ( (raw_data - raw_data.min())/(raw_data.max()-raw_data.min()) )
# Standardization
def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))

In [255]:
for i in X_train.columns:
    X_train[i] = standardize(X_train[i])
    
for i in X_test.columns:    
    X_test[i] = standardize(X_test[i])

In [256]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [257]:
# Convert to array for neural network
X_train_array = np.asarray(X_train).astype('float32')
X_test_array = np.asarray(X_test).astype('float32')

y_train_array = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test_array = np.asarray(y_test).astype('float32').reshape((-1,1))


In [259]:
# Neural network
comparison_dict ={'model':[],'params': [],'score': []}
params={'hidden_layer_sizes': [(20,5), (25,10)], 
        'activation': [ 'logistic', 'tanh', 'relu'], 
        'solver': ['sgd', 'adam'], 
        'alpha': [2, 10, 20]} 
# 'alpha': np.logspace(2,20)

for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model_params = (hidden_layer_sizes, activation, solver, alpha )
                model = MLPRegressor(hidden_layer_sizes = hidden_layer_sizes,
                                      activation = activation, solver = solver, alpha = alpha, random_state = 1)
                model.fit(X_train_array, y_train_array)

                y_test_pred = model.predict(X_test_array)
                MSE = mean_squared_error(y_test_pred , y_test_array)
                
                model_score = math.sqrt(MSE)
                comparison_dict['model'].append('neural_network_classifier')
                comparison_dict['params'].append(model_params)
                comparison_dict['score'].append(model_score)
                
dfMLResults = pd.DataFrame({'Model': comparison_dict.get('model') ,
              'Param': comparison_dict.get('params') ,
              'Score': comparison_dict.get('score')})

In [260]:
dfMLResults.sort_values(by = ['Score']).head(5)

,Model,Param,Score
18,neural_network_classifier,"((25, 10), logistic, sgd, 2)",5.163751
0,neural_network_classifier,"((20, 5), logistic, sgd, 2)",5.209543
19,neural_network_classifier,"((25, 10), logistic, sgd, 10)",5.222547
1,neural_network_classifier,"((20, 5), logistic, sgd, 10)",5.278690
35,neural_network_classifier,"((25, 10), relu, adam, 20)",5.284907


In [279]:
# Candidates
# ((25, 10), logistic, sgd, 2)
# ((25, 10), relu, adam, 20)


model = MLPRegressor(hidden_layer_sizes = (25, 10),
                                      activation = 'relu', solver = 'adam', alpha = 2, random_state = 42)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

In [286]:
test['FP_total_pred'] = y_test_pred
test_FP = test[['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName',
       'FP_total', 'FP_total_pred']].reset_index(drop = True)
test_FP = test_FP.sort_values(by=['FP_total_pred'], ascending = False)

In [289]:
test_FP.to_csv("NN_Data/Pred/PassingPred_081122.csv", index = False)

<h2> 2. Receiving <h2>

<h5> Load Receiving <h5>

In [732]:
receiving = pd.read_csv("NN_Data/Defense/receiving_pbp_OPP_081022.csv")

<h5> Grouping <h5>

In [733]:
group = receiving.groupby(by=['team',"gameId", "playerId","playerName", 'oppTeamId','nickName']).sum()

group.reset_index(inplace=True)
group = group[['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName','FP_total']]

<h5> Load RouteType <h5>

In [727]:
#receivingRoute = pd.read_csv("Split/Receiving_RouteType_0817.csv")
receivingRoute = pd.read_csv("Split/Receiving_RouteType_2020_0817.csv")
receivingRoute['route'] = receivingRoute['route'].astype(int)
targetType = np.unique(receivingRoute['route'])

for i in targetType:
    a = receivingRoute[receivingRoute['route'] == i][['playerName','FP/routes']].reset_index(drop = True)
    a = a.rename(columns={'FP/routes' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'] , how = 'left')
    

<h5> Load DropType <h5>

In [728]:
#receivingDrop = pd.read_csv("Split/Receiving_DropType_0817.csv")
receivingDrop = pd.read_csv("Split/Receiving_DropType_2020_0817.csv")
dropType = np.unique(receivingDrop['DropType_descr'])

for i in dropType:

    a = receivingDrop[receivingDrop['DropType_descr'] == i][['playerName','FP/routes']].reset_index(drop = True)
    a = a.rename(columns={'FP/routes' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'] , how = 'left')
    

<h5> Load Coverage Types <h5>

In [729]:
#receivingCoverage = pd.read_csv("Split/Receiving_CoverageType_0817.csv")
receivingCoverage = pd.read_csv("Split/Receiving_CoverageType_2020_0817.csv")
coverageType = np.unique(receivingCoverage['CoverageType_descr'])

for i in coverageType:

    a = receivingCoverage[receivingCoverage['CoverageType_descr'] == i][['playerName','FP/routes']].reset_index(drop = True)
    a = a.rename(columns={'FP/routes' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'] , how = 'left')
    

<h5> Load Defense<h5>

In [734]:
#receivingRoute_def = pd.read_csv("Split/Receiving_RouteType_def_0817.csv")
receivingRoute_def = pd.read_csv("Split/Receiving_RouteType_def_2020_0817.csv")
receivingRoute_def['route'] = receivingRoute_def['route'].astype(int)
targetType = np.unique(receivingRoute_def['route'])

for i in targetType:
    a = receivingRoute_def[receivingRoute_def['route'] == i][['nickName','FP/routes']].reset_index(drop = True)
    split = str(i) + "_def"
    a = a.rename(columns={'FP/routes' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

    

In [735]:
#receivingDrop_def = pd.read_csv("Split/Receiving_DropType_def_0817.csv")
receivingDrop_def = pd.read_csv("Split/Receiving_DropType_def_2020_0817.csv")
receivingDrop_def = receivingDrop_def.rename(columns = {'Team' : 'nickName'})
dropType = np.unique(receivingDrop_def['DropType_descr'])


for i in dropType:
    a = receivingDrop_def[receivingDrop_def['DropType_descr'] == i][['nickName','FP/routes']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/routes' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

In [736]:
#receivingCoverage_def = pd.read_csv("Split/Receiving_CoverageType_def_0817.csv")
receivingCoverage_def = pd.read_csv("Split/Receiving_CoverageType_def_2020_0817.csv")
coverageType = np.unique(receivingCoverage_def['CoverageType_descr'])


for i in coverageType:
    a = receivingCoverage_def[receivingCoverage_def['CoverageType_descr'] == i][['nickName','FP/routes']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/routes' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

In [697]:
# Merge with position
roster = pd.read_csv("roster.csv", index_col = False)

group = pd.merge(group, roster[['playerId', 'positionCategory']], left_on = ['playerId'],  #2
                          right_on=['playerId'], how = 'left')

In [698]:
group = group[(group['positionCategory'] == 'WR') | (group['positionCategory'] == 'TE')].reset_index(drop = True)

<h3> Train-Test split (same as passing)<h3>

In [25]:
groupSplit = group.copy()

train, test = train_test_split(groupSplit, test_size=0.2)

trainNum = train.drop(columns = ['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName'])
testNum = test.drop(columns = ['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName'])

X_train = trainNum.drop(['FP_total'], axis = 1)
y_train = trainNum['FP_total']
X_test = testNum.drop(['FP_total'], axis = 1)
y_test = testNum['FP_total']

for i in X_train.columns:
    X_train[i] = standardize(X_train[i])
    
for i in X_test.columns:    
    X_test[i] = standardize(X_test[i])
    
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Convert to array for neural network
X_train_array = np.asarray(X_train).astype('float32')
X_test_array = np.asarray(X_test).astype('float32')

y_train_array = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test_array = np.asarray(y_test).astype('float32').reshape((-1,1))

# Neural network
comparison_dict ={'model':[],'params': [],'score': []}
params={'hidden_layer_sizes': [(20,5), (25,10)], 
        'activation': [ 'logistic', 'tanh', 'relu'], 
        'solver': ['sgd', 'adam'], 
        'alpha': [2, 10, 20]} 
# 'alpha': np.logspace(2,20)

for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model_params = (hidden_layer_sizes, activation, solver, alpha )
                model = MLPRegressor(hidden_layer_sizes = hidden_layer_sizes,
                                      activation = activation, solver = solver, alpha = alpha, random_state = 1)
                model.fit(X_train_array, y_train_array)

                y_test_pred = model.predict(X_test_array)
                MSE = mean_squared_error(y_test_pred , y_test_array)
                
                model_score = math.sqrt(MSE)
                comparison_dict['model'].append('neural_network_classifier')
                comparison_dict['params'].append(model_params)
                comparison_dict['score'].append(model_score)
                
dfMLResults = pd.DataFrame({'Model': comparison_dict.get('model') ,
              'Param': comparison_dict.get('params') ,
              'Score': comparison_dict.get('score')})

NameError: name 'standardize' is not defined

In [344]:
# Candidate
# ((20, 5), relu, adam, 20)


model = MLPRegressor(hidden_layer_sizes = (20, 5),
                                      activation = 'relu', solver = 'adam', alpha = 20, random_state = 42)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

In [345]:
test['FP_total_pred'] = y_test_pred
test_FP = test[['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName',
       'FP_total', 'FP_total_pred']].reset_index(drop = True)
test_FP = test_FP.sort_values(by=['FP_total_pred'], ascending = False)

In [382]:
test_FP.to_csv("NN_Data/Pred/ReceivingPred_081122.csv", index = False)

<h2> 3. Rushing <h2>

In [745]:
rushing = pd.read_csv("NN_Data/Defense/rushing_pbp_OPP_081022.csv")

In [746]:
group = rushing.groupby(by=['team',"gameId", "playerId","playerName", 'oppTeamId','nickName']).sum()

group.reset_index(inplace=True)
group = group[['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName','FP_total']]

In [741]:
# Blocking
#rushingBlocking = pd.read_csv("Split/Rushing_BlockingScheme_0817.csv")
rushingBlocking = pd.read_csv("Split/Rushing_BlockingScheme_2020_0817.csv")
blockingType = np.unique(rushingBlocking['blockingSchemeType_descr'])

for i in blockingType:

    a = rushingBlocking[rushingBlocking['blockingSchemeType_descr'] == i][['playerName','FP/Touch']].reset_index(drop = True)
    a = a.rename(columns={'FP/Touch' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'] , how = 'left')
    

In [743]:
# Direction
#rushingDirection = pd.read_csv("Split/Rushing_direction_0817.csv")
rushingDirection = pd.read_csv("Split/Rushing_direction_2020_0817.csv")

directionType = np.unique(rushingDirection['RunDirection_descr'])

for i in directionType:

    a = rushingDirection[rushingDirection['RunDirection_descr'] == i][['playerName','FP/Touch']].reset_index(drop = True)
    a = a.rename(columns={'FP/Touch' : i})
    group = pd.merge(group, a, left_on = ['playerName'], right_on=['playerName'] , how = 'left')
    

In [747]:
# Defense - blocking
#rushingBlocking_def = pd.read_csv("Split/Rushing_Blocking_def_0817.csv")
rushingBlocking_def = pd.read_csv("Split/Rushing_Blocking_def_2020_0817.csv")

blockingType = np.unique(rushingBlocking_def['blockingSchemeType_descr'])

for i in blockingType:
    a = rushingBlocking_def[rushingBlocking_def['blockingSchemeType_descr'] == i][['nickName','FP/Touch']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/Touch' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

In [749]:
# Defense - direction
#rushingDirection_def = pd.read_csv("Split/Rushing_Direction_def_0817.csv")
rushingDirection_def = pd.read_csv("Split/Rushing_Direction_def_2020_0817.csv")

direction = np.unique(rushingDirection_def['RunDirection_descr'])

for i in direction:
    a = rushingDirection_def[rushingDirection_def['RunDirection_descr'] == i][['nickName','FP/Touch']].reset_index(drop = True)
    split = i + "_def"
    a = a.rename(columns={'FP/Touch' : split})
    group = pd.merge(group, a, left_on = ['nickName'], right_on=['nickName'], how = 'left')
    

In [750]:
# Merge with position
roster = pd.read_csv("roster.csv", index_col = False)

group = pd.merge(group, roster[['playerId', 'positionCategory']], left_on = ['playerId'],  #2
                          right_on=['playerId'], how = 'left')

In [751]:
group = group[(group['positionCategory'] == 'RB')].reset_index(drop = True)

<h5> Split and train (same as above) <h5>

In [373]:
groupSplit = group.copy()

train, test = train_test_split(groupSplit, test_size=0.2)

trainNum = train.drop(columns = ['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName'])
testNum = test.drop(columns = ['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName'])

X_train = trainNum.drop(['FP_total'], axis = 1)
y_train = trainNum['FP_total']
X_test = testNum.drop(['FP_total'], axis = 1)
y_test = testNum['FP_total']

for i in X_train.columns:
    X_train[i] = standardize(X_train[i])
    
for i in X_test.columns:    
    X_test[i] = standardize(X_test[i])
    
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

# Convert to array for neural network
X_train_array = np.asarray(X_train).astype('float32')
X_test_array = np.asarray(X_test).astype('float32')

y_train_array = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test_array = np.asarray(y_test).astype('float32').reshape((-1,1))

# Neural network
comparison_dict ={'model':[],'params': [],'score': []}
params={'hidden_layer_sizes': [(20,5), (25,10)], 
        'activation': [ 'logistic', 'tanh', 'relu'], 
        'solver': ['sgd', 'adam'], 
        'alpha': [2, 10, 20]} 
# 'alpha': np.logspace(2,20)

for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model_params = (hidden_layer_sizes, activation, solver, alpha )
                model = MLPRegressor(hidden_layer_sizes = hidden_layer_sizes,
                                      activation = activation, solver = solver, alpha = alpha, random_state = 1)
                model.fit(X_train_array, y_train_array)

                y_test_pred = model.predict(X_test_array)
                MSE = mean_squared_error(y_test_pred , y_test_array)
                
                model_score = math.sqrt(MSE)
                comparison_dict['model'].append('neural_network_classifier')
                comparison_dict['params'].append(model_params)
                comparison_dict['score'].append(model_score)
                
dfMLResults = pd.DataFrame({'Model': comparison_dict.get('model') ,
              'Param': comparison_dict.get('params') ,
              'Score': comparison_dict.get('score')})

In [375]:
# Candidates
#((25, 10), relu, adam, 20)

model = MLPRegressor(hidden_layer_sizes = (25, 10),
                                      activation = 'relu', solver = 'adam', alpha = 20, random_state = 42)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

In [377]:
test['FP_total_pred'] = y_test_pred
test_FP = test[['team', 'gameId', 'playerId', 'playerName', 'oppTeamId', 'nickName',
       'FP_total', 'FP_total_pred']].reset_index(drop = True)
test_FP = test_FP.sort_values(by=['FP_total_pred'], ascending = False)

In [381]:
test_FP.to_csv("NN_Data/Pred/RushingPred_081122.csv", index = False)